In [23]:
import os
#os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'

os.environ['TF_USE_LEGACY_KERAS'] = '1';

In [24]:
input_default_json_conf_file='fcnn_emotion4_testing_delay_default.json';

In [25]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
import json
from tensorflow.keras.preprocessing.image import load_img

In [26]:
import sys
sys.path.append('../library');

In [27]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [28]:
## Load json conf file
fd = open(os.path.join('./',input_default_json_conf_file));
DATA = json.load(fd);
fd.close()

# Variable globales

In [29]:
## Seed for the random variables
seed_number=0;

## Dataset 
dataset_base_dir     = DATA['dataset_base_dir'];
dataset_labels_file  = DATA['dataset_labels_file'];
dataset_name         = DATA['dataset_name'];

target_labels=['negative','neutro','pain','positive'];

## Model filepath
best_model_file = DATA['model_file'];

## times to count time
times = DATA['times'];

NCOD=DATA['ncod'];

## Output
output_base_dir = DATA['output_base_dir'];

##############################################

# If command line

In [30]:
for n in range(len(sys.argv)):
    if   sys.argv[n]=='--dataset-dir':
        dataset_base_dir=sys.argv[n+1];
    elif sys.argv[n]=='--dataset-file':
        dataset_labels_file=sys.argv[n+1];
    elif sys.argv[n]=='--dataset-name':
        dataset_name=sys.argv[n+1];
    elif sys.argv[n]=='--ncod':
        NCOD=int(sys.argv[n+1]);
    elif sys.argv[n]=='--model-file':
        best_model_file=sys.argv[n+1];
    elif sys.argv[n]=='--times':
        times=int(sys.argv[n+1]);
    elif sys.argv[n]=='--output-dir':
        output_base_dir=sys.argv[n+1];


print('   dataset_base_dir:',dataset_base_dir);
print('dataset_labels_file:',dataset_labels_file);
print('       dataset_name:',dataset_name);
print('               NCOD:',NCOD);
print('    best_model_file:',best_model_file);
print('              times:',times);
print('    output_base_dir:',output_base_dir);



   dataset_base_dir: /media/fernando/Expansion/DATASET/TESE/BER/BER2024/BER2024-FUSION/ber2024-source/ncod20_efficientnet_b3_efficientnet_b3_step1
dataset_labels_file: test.csv
       dataset_name: ber2024-fusion
               NCOD: 8
    best_model_file: /media/fernando/Expansion/OUTPUTS/DOCTORADO2/fcnn_emotion4_fusion_1/ber2024-fusion/training_validation_holdout/cls_ncod8/model_ncod8.h5
              times: 10
    output_base_dir: /media/fernando/Expansion/OUTPUTS/DOCTORADO2/fcnn_emotion4_fusion_1


# Set seed of random variables


In [31]:
np.random.seed(seed_number)
tf.keras.utils.set_random_seed(seed_number);

# Loading data of dataset

In [32]:
# Load filenames and labels
test_df = pd.read_csv(os.path.join(dataset_base_dir,dataset_labels_file));
print(test_df)

             d0        d1        d2        d3            d4            d5  \
0      0.000686  0.998839  0.000349  0.000126  1.989638e-05  9.997572e-01   
1      0.999747  0.000154  0.000078  0.000022  9.999405e-01  5.823886e-05   
2      0.000036  0.000102  0.000191  0.999670  5.370393e-07  1.223157e-06   
3      0.007865  0.991319  0.000236  0.000579  5.689665e-02  9.430852e-01   
4      0.000047  0.000282  0.000154  0.999517  6.336613e-06  7.395937e-05   
...         ...       ...       ...       ...           ...           ...   
16653  0.000441  0.999499  0.000023  0.000037  7.283359e-05  9.999247e-01   
16654  0.006311  0.993072  0.000470  0.000147  2.958973e-03  9.970395e-01   
16655  0.000197  0.000046  0.999420  0.000337  2.487195e-05  7.110825e-07   
16656  0.198577  0.799782  0.001604  0.000037  6.360198e-02  9.363549e-01   
16657  0.000017  0.000121  0.000156  0.999707  9.749557e-03  2.710924e-02   

                 d6            d7        d8        d9       d10       d11  

# Creating output directory

In [33]:

output_dir = os.path.join(output_base_dir,dataset_name,'test_delay','cls_ncod'+str(NCOD));

os.makedirs(output_dir,exist_ok = True);

# Create new model

In [34]:
import FusionEmotion4Lib.lib_model as mpp

model = mpp.create_model(load_weights=False,file_of_weight=best_model_file,ncod=NCOD);
model.summary()

mpp.save_model_parameters(model, os.path.join(output_dir,'parameters_stats.m'));


Loading the weights in: /media/fernando/Expansion/OUTPUTS/DOCTORADO2/fcnn_emotion4_fusion_1/ber2024-fusion/training_validation_holdout/cls_ncod8/model_ncod8.h5
Loaded the weights in: /media/fernando/Expansion/OUTPUTS/DOCTORADO2/fcnn_emotion4_fusion_1/ber2024-fusion/training_validation_holdout/cls_ncod8/model_ncod8.h5
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 8)                 104       
                                                                 
 dense_3 (Dense)             (None, 4)                 36        
                                                                 
Total params: 140 (560.00 Byte)
Trainable params: 140 (560.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


# Creating numpy vector

In [39]:
import pandas as pd

np_vector=[];
labelid_vector=[]
for index, row in test_df.iterrows():
    vec=row.iloc[:12].values;
    if vec.dtype != np.float32:
        vec = vec.astype(np.float32);
    np_vector.append(vec)
    labelid_vector.append(target_labels.index(row.iloc[12].lower()));

print(np_vector[0])
print(labelid_vector[0])

[6.8554503e-04 9.9883920e-01 3.4933464e-04 1.2598086e-04 1.9896381e-05
 9.9975723e-01 8.7311208e-05 1.3551982e-04 6.9812173e-01 1.7835021e-01
 9.7384840e-02 2.6143270e-02]
1


# Time

In [40]:
from pytictoc import TicToc
t = TicToc() 

# Evaluate best model

In [ ]:

from tqdm import tqdm

Count=0;
L=len(np_vector);
print('L:',L)


t.tic();
for m in range(times):
    print('m:',m)
    for n in tqdm(range(L)):
        res=mpp.evaluate_model_from_npvector(model, np_vector[n])
        if labelid_vector[n]==res:
            Count=Count+1;
t0=t.tocvalue();


results = dict();
results['categorical_accuracy']=Count*1.0/(L*times);
results['delayms']=t0*1000.0/(times*L);
results['number_of_parameters']=mpp.get_model_parameters(model);
results['ncod']=NCOD;

print(json.dumps(results, indent=4))


L: 16658
m: 0


 98%|████████████████████████████████████▏| 16304/16658 [17:57<00:27, 13.08it/s]

In [ ]:
# final all json
with open(os.path.join(output_dir,"testing_data_results.json"), 'w') as f:
    json.dump(results, f,indent=4);

tf.keras.backend.clear_session();
